In [1]:
# Importar as bibliotecas
import pandas as pd
from src.paths import DATA_INPUTS_OUTPUTS

pd.set_option('display.max_columns', 366)
%load_ext autoreload
%autoreload 2

In [2]:
%config Completer.use_jedi = False

In [3]:
# Carregar dados
Order_rating_DB = pd.read_excel(DATA_INPUTS_OUTPUTS / 'Data Analyst _ Zé Delivery Data Base.xlsx', sheet_name='Order rating DB')
Bad_Rating_reasons = pd.read_excel(DATA_INPUTS_OUTPUTS / 'Data Analyst _ Zé Delivery Data Base.xlsx', sheet_name='Bad Rating reasons')

In [4]:
# Dados sobre o dataset
Order_rating_DB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11259 entries, 0 to 11258
Data columns (total 13 columns):
Order ID               11259 non-null int64
Order Number           11259 non-null int64
Week                   11259 non-null int64
Date & Time            11259 non-null datetime64[ns]
Comment                127 non-null object
User                   11257 non-null object
Merchandise Revenue    11259 non-null float64
Payment Method         5990 non-null object
POC ID                 11259 non-null int64
Rating                 5134 non-null float64
Reasons                0 non-null float64
Shipping Type          11259 non-null object
State                  11239 non-null object
dtypes: datetime64[ns](1), float64(3), int64(4), object(5)
memory usage: 1.1+ MB


- Vamos verificar se há registros repetidos para cada campo
- Há dois Users faltando

In [5]:
Order_rating_DB.nunique()

Order ID               11259
Order Number           11259
Week                      45
Date & Time            11222
Comment                  114
User                    7600
Merchandise Revenue     2755
Payment Method             5
POC ID                    51
Rating                     6
Reasons                    0
Shipping Type              3
State                     44
dtype: int64

- Parece que há clientes que compraram mais de uma vez. Podemos fazer uma análise sobre isso depois.
- Uma análise sobre o tipo de pagamento também pode ser feita, porém, há muito registris nulos (assim como em Rating).
- Temos muitos campos com estado, uma análise sob este prisma também pode ser feita posteriormente.

In [6]:
# Registro com user faltando
Order_rating_DB[Order_rating_DB['User'].isna()]

,Order ID,Order Number,Week,Date & Time,Comment,User,Merchandise Revenue,Payment Method,POC ID,Rating,Reasons,Shipping Type,State
5604,96451,52519903,29,2017-07-14 16:37:49,NaN,NaN,7.416,Dinheiro,21,NaN,NaN,receber,Sao Paulo
5607,96477,83205118,29,2017-07-14 17:18:54,NaN,NaN,7.656,Dinheiro,66,NaN,NaN,receber,Sao Paulo


Não parece seguro seguir a análise com estes dois registros. Como não sabemos o que pode levar a ausência deste campo, vamos removê-lo.

In [7]:
Order_rating_DB.drop(index=[5604,5607], inplace=True)

In [8]:
# Vamos verificar se o número da semana está correto
[Order_rating_DB['Week'], Order_rating_DB['Date & Time'].dt.week]

[0         1
 1         1
 2         1
 3         1
 4         1
 5         1
 6         1
 7         2
 8         2
 9         2
 10        2
 11        2
 12        2
 13        2
 14        2
 15        2
 16        2
 17        2
 18        2
 19        2
 20        2
 21        2
 22        2
 23        2
 24        2
 25        2
 26        2
 27        2
 28        2
 29        2
          ..
 11229    45
 11230    45
 11231    45
 11232    45
 11233    45
 11234    45
 11235    45
 11236    45
 11237    45
 11238    45
 11239    45
 11240    45
 11241    45
 11242    45
 11243    45
 11244    45
 11245    45
 11246    45
 11247    45
 11248    45
 11249    45
 11250    45
 11251    45
 11252    45
 11253    45
 11254    45
 11255    45
 11256    45
 11257    45
 11258    45
 Name: Week, Length: 11257, dtype: int64,
 0        52
 1        52
 2        52
 3        52
 4        52
 5        52
 6        52
 7         1
 8         1
 9         1
 10        1
 11        1
 12      

Parece que há um pequeno problema com a primeira semana do ano. Ela não faz parte da semana 1, mas da semana 52. Vamos criar um novo campo com a semana correta.

In [9]:
Order_rating_DB['CorrectWeek'] = Order_rating_DB['Date & Time'].dt.week

In [10]:
# Modo de pagamento
Order_rating_DB['Payment Method'].unique()

array([nan, 'Cartão de Débito', 'Cartão de Crédito', 'Dinheiro', 'Débito',
       'Crédito'], dtype=object)

In [11]:
# Parece não estar padronizado. Vamos padronizar
Order_rating_DB['Payment Method'] = (Order_rating_DB['Payment Method']
                                     .replace('Cartão de Crédito', 'Crédito')
                                     .replace('Cartão de Débito', 'Débito')
                                    )
Order_rating_DB['Payment Method'].unique()

array([nan, 'Débito', 'Crédito', 'Dinheiro'], dtype=object)

Vamos olhar o Shipping Type:

In [12]:
Order_rating_DB['Shipping Type'].unique()

array(['receber', 'retirar', 'agendar'], dtype=object)

In [13]:
# O que significa o tipo de registro agendar? Vamos ver quanto representa cada um no total.
Order_rating_DB['Shipping Type'].value_counts()

receber    9881
retirar    1349
agendar      27
Name: Shipping Type, dtype: int64

In [14]:
# Vamos olha melhor ests 27 casos
Order_rating_DB[Order_rating_DB['Shipping Type'] == 'agendar'].sample(5)

,Order ID,Order Number,Week,Date & Time,Comment,User,Merchandise Revenue,Payment Method,POC ID,Rating,Reasons,Shipping Type,State,CorrectWeek
3958,224206,46885,23,2017-06-03 17:32:47,NaN,user13851gmail.com,29.560,NaN,58,NaN,NaN,agendar,PR,22
643,196705,19381,8,2017-02-17 12:56:24,NaN,user623gmail.com,210.240,NaN,21,5.0,NaN,agendar,SP,7
175,192837,15508,4,2017-01-17 07:50:55,NaN,user623gmail.com,210.240,NaN,26,5.0,NaN,agendar,SP,3
3253,218430,41109,21,2017-05-18 10:58:38,NaN,user11843gmail.com,373.824,NaN,21,5.0,NaN,agendar,SP,20
4073,225115,47794,24,2017-06-06 12:56:34,NaN,user11157gmail.com,530.832,NaN,21,5.0,NaN,agendar,SP,23


Não sabemos exatamente que tipo de remessa é essa, mas por ora, não vamos remover estes registros.

Vamos olhar o estado

In [15]:
Order_rating_DB['State'].unique()

array(['SP', 'RJ', 'MG', 'CE', 'PR', 'Campinas', 'São Paulo', 'Fortaleza',
       'São José do Rio Preto', 'Rio de Janeiro', 'S',
       'São Bernardo do Campo', 'Diadema', 'Belo Horizonte', nan,
       'Santo André', 'Curitiba', 'Betim', 'São José dos Campos',
       'Nova Iguaçu', 'Niter', 'São Caetano do Sul', 'Nova Igua',
       'Niterói', 'Guarulhos', 'Embu', 'Poços de Caldas', 'Santo Andr',
       'Itaquaquecetuba', 'Belford Roxo', 'Mesquita', 'Contagem',
       'Colombo', 'undefined', 'Osasco', 'Carapicuíba', 'Embu das Artes',
       'Taboão da Serra', 'Ferraz de Vasconcelos', 'Pinhais',
       'São Carlos', 'Maracanaú', 'Barueri', 'Nova Lima'], dtype=object)

Parece que os dados de estado não estão padronizados :(... além disso, há campos nulos e 'undefined'. <p>
Vamos ver a participação de tipo de registro estado no total.

In [16]:
Order_rating_DB['State'].value_counts()

SP                       4104
São Paulo                2424
Rio de Janeiro           1083
RJ                        737
São Bernardo do Campo     419
undefined                 390
Belo Horizonte            345
Nova Iguaçu               281
MG                        277
Campinas                  229
Diadema                   147
S                         112
Curitiba                   81
CE                         77
PR                         74
Fortaleza                  73
São José do Rio Preto      55
Betim                      53
Santo André                50
Belford Roxo               48
São Caetano do Sul         38
Contagem                   26
Mesquita                   18
Osasco                     17
Guarulhos                  15
Embu das Artes             12
São José dos Campos        10
Embu                        8
Taboão da Serra             7
Itaquaquecetuba             6
Colombo                     5
Poços de Caldas             3
Pinhais                     2
Niter     

In [17]:
# Vamos tentar normalizar este campo.
Order_rating_DB['CorrectState'] = (Order_rating_DB['State']
                                     .map(
                                         {'SP':'SP',
                                          'São Paulo':'SP',
                                          'Rio de Janeiro':'RJ',
                                          'RJ':'RJ',
                                          'São Bernardo do Campo':'SP',
                                          'undefined':'undefined',
                                          'Belo Horizonte':'MG',
                                          'Nova Iguaçu':'RJ',
                                          'MG':'MG',
                                          'Campinas':'SP',
                                          'Diadema':'SP',
                                          'S':'SP',
                                          'Curitiba':'PR',
                                          'CE':'CE',
                                          'PR':'PR',
                                          'Fortaleza':'CE',
                                          'São José do Rio Preto':'SP',
                                          'Betim':'MG',
                                          'Santo André':'SP',
                                          'Belford Roxo':'RJ',
                                          'São Caetano do Sul':'SP',
                                          'Contagem':'MG',
                                          'Mesquita':'RJ',
                                          'Osasco':'SP',
                                          'Guarulhos':'SP',
                                          'Embu das Artes':'SP',
                                          'São José dos Campos':'SP',
                                          'Embu':'SP',
                                          'Taboão da Serra':'SP',
                                          'Itaquaquecetuba':'SP',
                                          'Colombo':'PR',
                                          'Poços de Caldas':'MG',
                                          'Pinhais':'PR',
                                          'Niter':'RJ',
                                          'São Carlos':'SP',
                                          'Ferraz de Vasconcelos':'SP',
                                          'Maracanaú':'CE',
                                          'Carapicuíba':'SP',
                                          'Barueri':'SP',
                                          'Nova Lima':'MG',
                                          'Niterói':'RJ',
                                          'Santo Andr':'SP',
                                          'Nova Igua':'RJ'}
                                     )
                                    )

In [18]:
Order_rating_DB['CorrectState'].unique()

array(['SP', 'RJ', 'MG', 'CE', 'PR', nan, 'undefined'], dtype=object)

In [19]:
Order_rating_DB['CorrectState'].value_counts()

SP           7658
RJ           2171
MG            705
undefined     390
PR            162
CE            151
Name: CorrectState, dtype: int64

In [20]:
# Vamos remover os espaços os nomes das colunas
columns_name = [x.replace(' ', '') for x in list(Order_rating_DB.columns)]
Order_rating_DB.columns = columns_name

Agora, vamos salvar estes dados E avançar na análise a partir deles.

In [21]:
Order_rating_DB.to_csv(DATA_INPUTS_OUTPUTS / 'OrderRatingDB.csv', index=False)